#### Imports

In [ ]:
from scipy import spatial, sparse
from scipy.stats import chi2
from collections import Counter
from num2words import num2words
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.externals import joblib 
from nltk.tokenize import word_tokenize
from sklearn.model_selection import KFold
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import os
import imp
import gzip
import copy
import nltk
import pickle
import scipy
import string
import gensim
import operator
import datetime

import numpy as np
import pandas as pd
import LDA_ELJST as lda
import matplotlib.pyplot as plt

In [ ]:
import utils as my_utils
from sentiment import SentimentAnalysis

### Read Data

In [ ]:
dataset_name = "amazon_musical"

In [ ]:
dataset = pd.read_pickle("datasets/datadf_amazon_musical")

In [ ]:
dataset.head(2)

In [ ]:
count_matrix, _, vocabulary, words = my_utils.processReviews(dataset['text'].values)

In [ ]:
Counter(dataset['sentiment'])

In [ ]:
barren = np.where(count_matrix.sum(1)==0)[0]

In [ ]:
barren

In [ ]:
ratings = dataset['sentiment'].values

### Making Edge_Dict

In [ ]:
dataset_name

In [ ]:
# docs_edges = pickle.load(open("resources/amazon_musical_bert_0.8.pickle","rb"))

In [ ]:
# docs_edges = np.delete(docs_edges, barren).tolist()

## Run Model

In [ ]:
maxiter = 20
lambda_param = 10.0
n_topics = 10
n_sentiment = 5

In [ ]:
dataset_name

In [ ]:
folder_name = str(datetime.datetime.now()) + "_" + dataset_name
os.mkdir("dumps/"+folder_name)

In [ ]:
edge_list = ["resources/amazon_musical_bert_0.2.pickle",
             "resources/amazon_musical_bert_0.3.pickle",
             "resources/amazon_musical_bert_0.4.pickle",
             "resources/amazon_musical_bert_0.5.pickle",
             "resources/amazon_musical_fasttext_0.2.pickle",
             "resources/amazon_musical_fasttext_0.3.pickle",
             "resources/amazon_musical_fasttext_0.4.pickle",
             "resources/amazon_musical_fasttext_0.5.pickle",
             "resources/amazon_musical_glove_0.2.pickle",
             "resources/amazon_musical_glove_0.3.pickle",
             "resources/amazon_musical_glove_0.4.pickle",
             "resources/amazon_musical_glove_0.5.pickle"]

In [ ]:
import multiprocessing

In [ ]:
def multiprocessing_func(edge_loc):
    docs_edges = pickle.load(open(edge_loc,"rb"))
    sampler = lda.LdaSampler(count_matrix, ratings, docs_edges, words, vocabulary,
                         n_sentiment = n_sentiment, n_topics=n_topics, lambda_param=lambda_param)

    print "Running Sampler...", edge_loc
    sampler.run(maxiter=maxiter)
    joblib.dump(sampler, "dumps/" + folder_name + "/sampler_" + edge_loc)

In [ ]:
pool = multiprocessing.Pool()

In [ ]:
pool.map(multiprocessing_func, topics_grid)

In [ ]:
docs_edges = pickle.load(open(edge_list[3],"rb"))

In [ ]:
sampler = lda.LdaSampler(count_matrix[:100], ratings[:100], docs_edges[:100], words, vocabulary,
                         n_sentiment = n_sentiment, n_topics=4, lambda_param=lambda_param)

In [ ]:
sampler.phi()[0,:,0].argsort()[-K:]